In [4]:
import requests
import pandas as pd
import csv
from bs4 import BeautifulSoup as BS, SoupStrainer as SS
import urllib2
import requests
from unidecode import unidecode
from lxml.html.clean import Cleaner
import re
import tldextract

In [24]:
from scrapy import Item, Field
from scrapy.loader import ItemLoader
from scrapy.loader.processors import TakeFirst, Identity


class Detail(Item):
    detail_url = Field()
    name = Field()
    address = Field()
    url = Field()
    about = Field()

class DetailLoader(ItemLoader):
    default_output_processor = TakeFirst()
    address_out = Identity()

In [21]:
cd ces2017

C:\Users\joshu\lux-python\scrappy-scrapin\ces2017\ces2017


In [22]:
!ls

__init__.py
__init__.pyc
items.py
items.pyc
middlewares.py
pipelines.py
settings.py
settings.pyc
spiders


In [25]:
import scrapy
from selenium import webdriver
# from ces2017.items import Detail, DetailLoader

def safe_strip(x):
    return x if x is None else x.strip()

class CesSpider(scrapy.Spider):
    name = 'ces'

    download_delay = 1

    def __init__(self):
        self.browser = webdriver.PhantomJS()
        self.start = 'http://ces17.mapyourshow.com/7_0/alphalist.cfm?alpha=*'
        self.items = []

    def start_requests(self):
        yield scrapy.Request(self.start, callback=self.parse_start)

    def parse_start(self, response):
        self.browser.get(response.url)
        items = self.browser.find_elements_by_css_selector('td.mys-table-exhname a')

        while len(items) > len(self.items):
            for item in items:
                detail_url = item.get_attribute('href')
                yield scrapy.Request(detail_url, callback=self.parse_detail)

            self.items = items

            # scroll down and update list of items
            self.browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            items = self.browser.find_elements_by_css_selector('td.mys-table-exhname a')
            print '>>>>> scrolling: {} --> {}'.format(len(self.items), len(items))

        self.browser.quit()

In [27]:
!scrapy crawl ces2017 -o out.json

2016-12-16 11:29:26 [scrapy] INFO: Scrapy 1.2.2 started (bot: ces2017)
2016-12-16 11:29:26 [scrapy] INFO: Overridden settings: {'NEWSPIDER_MODULE': 'ces2017.spiders', 'FEED_URI': 'out.json', 'SPIDER_MODULES': ['ces2017.spiders'], 'BOT_NAME': 'ces2017', 'ROBOTSTXT_OBEY': True, 'FEED_FORMAT': 'json'}
Traceback (most recent call last):
  File "c:\anaconda2\lib\runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "c:\anaconda2\lib\runpy.py", line 72, in _run_code
    exec code in run_globals
  File "C:\Anaconda2\Scripts\scrapy.exe\__main__.py", line 9, in <module>
  File "c:\anaconda2\lib\site-packages\scrapy\cmdline.py", line 142, in execute
    _run_print_help(parser, _run_command, cmd, args, opts)
  File "c:\anaconda2\lib\site-packages\scrapy\cmdline.py", line 88, in _run_print_help
    func(*a, **kw)
  File "c:\anaconda2\lib\site-packages\scrapy\cmdline.py", line 149, in _run_command
    cmd.run(args, opts)
  File "c:\anaconda2\lib\site-packages\s